In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

In [16]:
train = pd.read_csv('../data/train_Winsorization_시총_상장.csv', encoding='euc-kr')
test = pd.read_csv('../data/test_시총_상장.csv', encoding='euc-kr')

In [17]:
X_train_sum = train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR',
       '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률',
       '매출액영업이익률']]
y_train = train[['t-1감사의견코드']]
X_test_sum = test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR',
       '총자산대비영업현금흐름', '자기자본증가율', '총자본투자효율', '총자본순이익률',
       '매출액영업이익률']]
y_test = test[['t-1감사의견코드']]

In [18]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [19]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train_sum)
X_test_sc = scaler.transform(X_test_sum)

In [20]:
X_train_sc=pd.DataFrame(X_train_sc, columns=X_train_sum.columns)
X_test_sc=pd.DataFrame(X_test_sc, columns=X_test_sum.columns)

In [21]:
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import RandomUnderSampler

#
# OneSidedSelection 적용
undersampler = OneSidedSelection(random_state=42)
X_train_sc, y_train = undersampler.fit_resample(X_train_sc, y_train)

# 1:1 비율로 데이터셋을 다시 조정합니다.
custom_undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_train_sc, y_train = custom_undersampler.fit_resample(X_train_sc, y_train)

In [22]:

from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import RandomUnderSampler

# OneSidedSelection 적용
undersampler = OneSidedSelection(random_state=42)
X_test_sc, y_test = undersampler.fit_resample(X_test_sc, y_test)

# 1:1 비율로 데이터셋을 다시 조정합니다.
custom_undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_test_sc, y_test = custom_undersampler.fit_resample(X_test_sc, y_test)

In [23]:
X_train_sc=pd.concat([X_train_sc,y_train],axis=1)

In [24]:
X_test_sc=pd.concat([X_test_sc,y_test],axis=1)

In [25]:
X_train_sc['t-1감사의견코드'].value_counts()

0.0    90
1.0    90
Name: t-1감사의견코드, dtype: int64

In [26]:
X_test_sc['t-1감사의견코드'].value_counts()

0.0    27
1.0    27
Name: t-1감사의견코드, dtype: int64

In [27]:
X_train_sc.to_csv('train_상장_1대1.csv', index=False, encoding='euc-kr')
X_test_sc.to_csv('test_상장_1대1.csv', index=False, encoding='euc-kr')